In [ ]:
import psycopg2
import src.params as params
from faker import Faker
import random
from geopy.geocoders import Nominatim
from geopy.point import Point
import math
import numpy as np


In [ ]:
def generador_direccion():

    geolocator = Nominatim(user_agent="durumkebab")

    centro = Point(40.4168, -3.7038)
    lat_madrid = math.radians(centro.latitude)
    lon_madrid = math.radians(centro.longitude)

    direccion_flag = True
    while direccion_flag:
        d = np.random.uniform(0, 10) * 1000
        ang = np.random.uniform(0, 2*np.pi)
        R = 6371000
        lat = math.asin(math.sin(lat_madrid) * math.cos(d/R) + math.cos(lat_madrid) * math.sin(d/R) * math.cos(ang))
        lon = lon_madrid + math.atan2(math.sin(ang) * math.sin(d/R) * math.cos(lat_madrid), math.cos(d/R) - math.sin(lat_madrid) * math.sin(lat))
        lat = math.degrees(lat)
        lon = math.degrees(lon)

        direccion = geolocator.reverse((lat, lon))
        try:
            int(direccion[0].split(",")[0])
            try:
                int(direccion[0].split(",")[1])
            except:
                latitud = direccion[1][0]
                longitud = direccion[1][1]
                direccion = "{}, {}, Madrid" \
                    .format(direccion[0].split(",")[1].strip(), 
                        direccion[0].split(",")[0])
                direccion_flag = False
        except:
            pass
    
    return direccion, latitud, longitud
        


In [ ]:
def generador_clientes(n_clientes_nuevos):

    try:
        conn = psycopg2.connect(
            host = params.host, 
            dbname = params.dbname, 
            user = params.user, 
            password = params.password, 
            port = params.port)
        cur = conn.cursor()
    except (Exception, psycopg2.Error) as error:
        print("Error en la apertura de base de datos: \n\t{}".format(error))
        return
    
    f = Faker('es_ES')
    i = 0
    n_errores = 0
    while i < n_clientes_nuevos:
        cliente = {}
        cliente["nombre"] = f.name()
        cliente["email"] = '{}@mail.com' \
            .format(cliente["nombre"].lower() \
            .replace(" ", ".")
            .replace("á", "a") \
            .replace("é", "e") \
            .replace("í", "i") \
            .replace("ó", "o") \
            .replace("ú", "u") \
            .replace("ü", "u"))
        
        cliente["direccion"], cliente["latitud"], \
            cliente["longitud"] = generador_direccion(geolocator)

        try:
            sql_query = """
                INSERT INTO 
                    clientes(nombre, email, direccion, latitud, longitud)
                VALUES
                    (%s, %s, %s, %s, %s);"""
            args = (cliente["nombre"], 
                    cliente["email"], 
                    cliente["direccion"], 
                    cliente["latitud"], 
                    cliente["longitud"])
            cur.execute(sql_query, args)
            conn.commit()
            i += 1
        except Exception:
            print("Error:")
            print(cliente["nombre"], 
                cliente["email"], 
                cliente["direccion"], 
                cliente["latitud"], 
                cliente["longitud"])
            conn.rollback()
            n_errores += 1

        try:
            sql_query = """
                    SELECT id FROM clientes WHERE nombre = %s;
                    """
            cur.execute(sql_query, (cliente["nombre"],))
            id = cur.fetchone()[0]

            sql_query = """
                INSERT INTO
                    usuarios (username, id_cliente, tipo, contraseña)
                VALUES
                    (%s, %s, %s, %s);"""
            args = (cliente["email"], id, 'cliente', '1234')
            cur.execute(sql_query, args)
            conn.commit()
        except Exception:
            print("Error:")
            print(cliente["nombre"], 
                cliente["email"], 
                cliente["direccion"], 
                cliente["latitud"], 
                cliente["longitud"])
            conn.rollback()
    
    cur.close()
    conn.close()

    return "{} errores en la inserción".format(n_errores)

        

In [ ]:
generador_clientes(0)

In [ ]:
def generador_repartidores(n_repartidores_nuevos):

    try:
        conn = psycopg2.connect(
            host = params.host, 
            dbname = params.dbname, 
            user = params.user, 
            password = params.password, 
            port = params.port)
        cur = conn.cursor()
    except (Exception, psycopg2.Error) as error:
        print("Error en la apertura de base de datos: \n\t{}".format(error))
        return
    
    f = Faker('es_ES')

    i = 0
    n_errores = 0
    while i < n_repartidores_nuevos:
        repartidor = {}
        repartidor["nombre"] = f.name()
        repartidor["email"] = '{}@mail.com' \
            .format(repartidor["nombre"].lower() \
            .replace(" ", ".") \
            .replace("á", "a") \
            .replace("é", "e") \
            .replace("í", "i") \
            .replace("ó", "o") \
            .replace("ú", "u") \
            .replace("ü", "u"))
        
        status_lst = [True, False]
        repartidor["status"] = random.choices(status_lst, weights = [0, 100])[0]

        vehiculos_lst = ["Bicicleta", "Motocicleta", "Patín"]
        repartidor["vehiculo"] = random.choices(vehiculos_lst, weights = [35, 55, 10])[0]

        try:
            sql_query = """
                INSERT INTO 
                    repartidores(nombre, email, status, vehiculo)
                VALUES
                    (%s, %s, %s, %s);"""
            args = (repartidor["nombre"], 
                    repartidor["email"], 
                    repartidor["status"], 
                    repartidor["vehiculo"])
            cur.execute(sql_query, args)
            conn.commit()
            i += 1
        except Exception:
            print("Error:")
            print(repartidor["nombre"], 
                repartidor["email"], 
                repartidor["status"], 
                repartidor["vehiculo"])
            conn.rollback()
            n_errores += 1
        
        try:
            sql_query = """
                    SELECT id FROM repartidores WHERE nombre = %s;
                    """
            cur.execute(sql_query, (repartidor["nombre"],))
            id = cur.fetchone()[0]

            sql_query = """
                INSERT INTO
                    usuarios (username, id_repartidor, tipo, contraseña)
                VALUES
                    (%s, %s, %s, %s);"""
            args = (repartidor["email"], id, 'repartidor', 'abcd')
            cur.execute(sql_query, args)
            conn.commit()
        except Exception:
            print("Error:")
            print(cliente["nombre"], 
                cliente["email"], 
                cliente["direccion"], 
                cliente["latitud"], 
                cliente["longitud"])
            conn.rollback()

    cur.close()
    conn.close()

    return "{} errores en la inserción".format(n_errores)


In [ ]:
generador_repartidores(0)

In [ ]:
def generador_admins(n_admins_nuevos):

    try:
        conn = psycopg2.connect(
            host = params.host, 
            dbname = params.dbname, 
            user = params.user, 
            password = params.password, 
            port = params.port)
        cur = conn.cursor()
    except (Exception, psycopg2.Error) as error:
        print("Error en la apertura de base de datos: \n\t{}".format(error))
        return
    
    f = Faker('es_ES')

    i = 0
    n_errores = 0
    while i < n_admins_nuevos:
        admin = {}
        admin["nombre"] = f.name()
        admin["email"] = '{}@mail.com' \
            .format(admin["nombre"].lower() \
            .replace(" ", ".")
            .replace("á", "a") \
            .replace("é", "e") \
            .replace("í", "i") \
            .replace("ó", "o") \
            .replace("ú", "u") \
            .replace("ü", "u"))

        try:
            sql_query = """
                INSERT INTO 
                    administradores(nombre, email)
                VALUES
                    (%s, %s);"""
            args = (admin["nombre"], 
                    admin["email"])
            cur.execute(sql_query, args)
            conn.commit()
            i += 1
        except Exception:
            print("Error:")
            print(admin["nombre"], 
                admin["email"])
            conn.rollback()
            n_errores += 1

        try:
            sql_query = """
                    SELECT id FROM administradores WHERE nombre = %s;
                    """
            cur.execute(sql_query, (admin["nombre"],))
            id = cur.fetchone()[0]

            sql_query = """
                INSERT INTO
                    usuarios (username, id_administrador, tipo, contraseña)
                VALUES
                    (%s, %s, %s, %s);"""
            args = (admin["email"], id, 'admin', 'qwerty')
            cur.execute(sql_query, args)
            conn.commit()
        except Exception:
            print("Error:")
            print(admin["nombre"], 
                admin["email"])
            conn.rollback()
    
    cur.close()
    conn.close()

    return "{} errores en la inserción".format(n_errores)

In [ ]:
generador_admins(0)

In [ ]:
import pandas as pd

comercios = pd.read_csv("restaurants.csv")
comercios = comercios.drop_duplicates(["name", "address"])
comercios = comercios.dropna(subset=["number"])

try:
    conn = psycopg2.connect(
        host = params.host, 
        dbname = params.dbname, 
        user = params.user, 
        password = params.password, 
        port = params.port)
    cur = conn.cursor()
except (Exception, psycopg2.Error) as error:
    print("Error en la apertura de base de datos: \n\t{}".format(error))

i=0
for index, rows in comercios.iterrows():
    try:
        rows["number"] = int(rows["number"].split(".")[0])
        sql_query = """
            INSERT INTO 
                comercios(nombre, zona, direccion, tipo, latitud, longitud)
            VALUES
                (%s, %s, %s, %s, %s, %s);"""
        args = (rows["name"], 
                rows["postal_code"], 
                "{}, {}, {}".format(rows["address"], int(rows["number"]), rows["city"]), 
                rows["type"],
                rows["latitude"],
                rows["longitude"])
        cur.execute(sql_query, args)
        conn.commit()
    except Exception:
        i += 1
        print(rows["id"])
        conn.rollback()

print(i)



In [ ]:
conn.rollback()

In [ ]:
cur.close()
conn.close()